<a href="https://colab.research.google.com/github/ALW-N/NNDL_LAB/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Preprocessing

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert class labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


Optional Data Augmentation

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)


Network Architecture Design


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Create the model
model = Sequential()

# Input layer
model.add(Flatten(input_shape=(32, 32, 3)))

# Hidden layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))

# Output layer with softmax activation
model.add(Dense(10, activation='softmax'))

# Model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 3072)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,573,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,707,274 (6.51 MB)

 Trainable params: 1,707,274 (6.51 MB)

 Non-trainable params: 0 (0.00 B)

Interpretation:


Flatten layer: Converts the 32x32x3 input image into a 1D vector of size 3072.


Hidden layers: Two fully connected layers with 512 and 256 neurons. The ReLU activation function is chosen to introduce non-linearity and avoid the vanishing gradient problem.

Output layer: A softmax layer with 10 neurons, corresponding to the 10 classes, and it outputs the probability distribution for each class.

Activation Functions:

ReLU (Rectified Linear Unit): ReLU is commonly used because it helps with faster convergence and prevents vanishing gradients. It outputs a value if it's positive; otherwise, it returns zero.

Tanh: Another activation function we could have used is tanh, which maps the output between -1 and 1, useful for symmetrically distributed inputs but prone to vanishing gradients.

Loss Function and Optimizer

In [4]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


Training the Model:

In [5]:
# Train the model with data augmentation
batch_size = 64
epochs = 50

history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_test, y_test),
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


781/781 ━━━━━━━━━━━━━━━━━━━━ 41s 49ms/step - accuracy: 0.2206 - loss: 2.1460 - val_accuracy: 0.3087 - val_loss: 1.8883
Epoch 2/50
  1/781 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.2344 - loss: 2.1542

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - accuracy: 0.2344 - loss: 2.1542 - val_accuracy: 0.3191 - val_loss: 1.8768
Epoch 3/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 35s 45ms/step - accuracy: 0.2838 - loss: 1.9490 - val_accuracy: 0.3484 - val_loss: 1.8324
Epoch 4/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step - accuracy: 0.2969 - loss: 2.0116 - val_accuracy: 0.3507 - val_loss: 1.8256
Epoch 5/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 35s 44ms/step - accuracy: 0.3079 - loss: 1.8922 - val_accuracy: 0.3577 - val_loss: 1.7833
Epoch 6/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - accuracy: 0.3125 - loss: 1.8756 - val_accuracy: 0.3560 - val_loss: 1.7838
Epoch 7/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 41s 46ms/step - accuracy: 0.3193 - loss: 1.8567 - val_accuracy: 0.3796 - val_loss: 1.7647
Epoch 8/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - accuracy: 0.2969 - loss: 2.0395 - val_accuracy: 0.3797 - val_loss: 1.7680
Epoch 9/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 34s 44ms/step - accuracy: 0.3257 - loss: 1.8466 - val_accurac

Model Evaluation:


In [6]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

# Confusion matrix, precision, recall, F1-score
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_pred = np.argmax(model.predict(x_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4277 - loss: 1.6273
Test accuracy: 0.429500013589859
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
[[345  45  97  41  21   5  38 101 115 192]
 [ 12 452  29  34  10  20  40  45  19 339]
 [ 51  25 281  85 168  37 171 135  15  32]
 [  3   9  88 232  83 165 238 103  18  61]
 [ 27   7 132  50 394  16 178 143  17  36]
 [  4   7 101 157 100 298 167 111  17  38]
 [  2   5  76  73 129  30 609  41   7  28]
 [  8  19  52  47  86  44  70 567   7 100]
 [ 53  75  21  40  21  23  29  23 449 266]
 [  9 123  11  38  12  12  48  62  17 668]]
              precision    recall  f1-score   support

           0       0.67      0.34      0.46      1000
           1       0.59      0.45      0.51      1000
           2       0.32      0.28      0.30      1000
           3       0.29      0.23      0.26      1000
           4       0.38      0.39      0.39      1000
           5       0.46      0.30      0.36      1000
           6       0.38      0.61      0.

Optimization Strategies:

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

# Train with callbacks
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_test, y_test),
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs,
                    callbacks=[early_stopping, lr_scheduler])


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


781/781 ━━━━━━━━━━━━━━━━━━━━ 45s 57ms/step - accuracy: 0.3822 - loss: 1.7079 - val_accuracy: 0.4365 - val_loss: 1.6111 - learning_rate: 0.0010
Epoch 2/50
  1/781 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.2969 - loss: 1.6712

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


781/781 ━━━━━━━━━━━━━━━━━━━━ 1s 713us/step - accuracy: 0.2969 - loss: 1.6712 - val_accuracy: 0.4385 - val_loss: 1.6082 - learning_rate: 0.0010
Epoch 3/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 79s 54ms/step - accuracy: 0.3796 - loss: 1.6980 - val_accuracy: 0.4475 - val_loss: 1.6017 - learning_rate: 0.0010
Epoch 4/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - accuracy: 0.4375 - loss: 1.5108 - val_accuracy: 0.4463 - val_loss: 1.6060 - learning_rate: 0.0010
Epoch 5/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 74s 45ms/step - accuracy: 0.3857 - loss: 1.6985 - val_accuracy: 0.4357 - val_loss: 1.5976 - learning_rate: 0.0010
Epoch 6/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.3906 - loss: 1.7101 - val_accuracy: 0.4370 - val_loss: 1.5982 - learning_rate: 0.0010
Epoch 7/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step - accuracy: 0.3861 - loss: 1.6957 - val_accuracy: 0.4440 - val_loss: 1.5910 - learning_rate: 0.0010
Epoch 8/50
781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - accuracy: 0.2812 - loss: 1.8850